### Piston model:

complex pressure *$P_t(p_z)$* at a point *$p_z$* due to a flat circular piston emitter at point *$p_t$*

### $P_t(p_z) = \frac{P_{ref}}{d(p_z, p_t)} \cdot \frac{2 \cdot J_1(k\cdot r\cdot \textrm{sin} \theta_{zt})}{k\cdot r\cdot \textrm{sin} \theta_{zt}}\cdot e^{i(\phi_t + k\cdot d(p_z,p_t))}$

$ P_{ref} $ is an arbitrary variabble defined by emitter amplitude, 

### $P_{ref} = A \cdot V_{pp} $

where $ A $ is the emitter output efficiency

and $ V_{pp} $ is the excitation signal peak-to-peak amplitude 

$ d(p_z, p_t) $ is the Euclidean distance between point $ p_z $ and the center of the emitter, $ p_t $ 

The directivity function for an emitter depends on the angle $\theta$ between the emitter normal and point $ p_z $ 

### $ D_f = \frac{2 \cdot J_1(k\cdot r\cdot \textrm{sin} \theta_{zt})} {k\cdot r\cdot \textrm{sin} \theta_{zt}} $

where $ J_1 $ is the Bessel function of the first kind, 

$ k=2\pi/\lambda $ is the wavenumber, 

$ r $ is the emitter radius, 

$ \theta_{zt} $ is the polar angle between points $p_z$ and $p_t$, 

$ \phi_t $ is the initial phase of the emitter



In [26]:
import sys
import math as math
import numpy as np

In [27]:
def rectangular_grid_to_coordinates( rows, columns, pitch ):
    
    num_points = columns * rows
    
    x_coords = np.linspace(-pitch * (columns - 1) / 2, 
                            pitch * (columns - 1) / 2, 
                            columns )
     
    y_coords = np.linspace(-pitch * (rows - 1) / 2, 
                            pitch * (rows - 1) / 2, 
                            rows )
    
    return np.array([ (x, y, 0.0) for x in x_coords for y in y_coords])

In [28]:
def hexagon_diagonal_to_coordinates( d, x_spacing=10.5/1000, y_spacing=9/1000 ) -> list((float, float, float)): 
    """
    Coordinate system for d-transducers diameter hexagon. Centrepoint of central transducer is at origin (0,0,0).
    Array begins with the bottom left transducer.
    
    args: 
        d:          diameter of hexagon (longest row) in transducer units 
        x_spacing:  interspacing between elements in the x axis
        y_spacing:  interspacing between elements in the y axis
        f_tran:     focal length of the PAT [m]
    
    returns:
        coords: nx3 array of coords for this hexagon, with [0, 0, 0] as the centrepoint.
    """
    
    # from the diameter in transducer units (central and longest row) calculate array with transducers count 
    # for bottom row up to central row:    
    bottom_to_central_row_tran_count = np.arange(np.floor((d+1)/2), np.floor(d+1), 1, dtype=int)

    # calculate array with rows' transducers count:
    rows_transducer_count = np.concatenate( (   bottom_to_central_row_tran_count,
                                                np.flip( bottom_to_central_row_tran_count )[1:]), 
                                            axis=0)
    coords = []   
    # for each row, depending on whether it is offset or not (i.e. shifted in relation to central row), 
    # calculate and assign X Y coordinates to each transducer:
    for row, row_length in enumerate(rows_transducer_count):
        for elem in range(row_length):      
            coord_x = x_spacing * ( elem - row_length/2 + .5 )
            coord_y = -sys.maxsize - 1
            coord_z = 0
            
            if d % 2 != 0:
                coord_y = y_spacing * (row - (d-1)/2)
            else:
                coord_y = y_spacing * (row - d/2)
                
            coords.append((coord_x, coord_y, coord_z))  
    
    return np.array(coords)

In [29]:
# reference sound pressure level for airborne sound is 20 micropascals (μPa or e-6) or 15.849 μPa
db_spl_to_pascal = lambda db: 10**((db-20)/20e-6)

pascal_to_db_spl = lambda pa: 20*math.log10(pa/20e-6)

In [34]:

rho_a = 1.184                           # density of air at 25°C (kg/m3) 
c = 346.13                              # speed of sound in air at 25°C (m/s)
f = 40000                               # frequency (Hz)

_lambda = lambda f: c/f                 # wavelength (m)

_k = lambda f: 2 * math.pi/_lambda(f)     # wavenumber 

# Euclidean distance, 2-norm or magnitude of the vector, sqrt of the inner product of a vector with itself 
_distance = lambda pz, pt: math.sqrt( (pz[0]-pt[0])**2 + (pz[1]-pt[1])**2 + (pz[2]-pt[2])**2 )  

# cross product of pz and pt over the distance between pz
_sin_theta = lambda pz, pt: math.sqrt( (pz[0]-pt[0])**2 + (pz[1]-pt[1])**2 ) / _distance(pz, pt)  

def directivity(sin_theta, k, r=4.5/1000):

    # argument of 1st order Bessel function
    bessel_J1 = k*r*sin_theta
    
    # taylor expansion of first order Bessel function over its agrument — J_1(bessel_J1)/bessel_J1
    # wolframalpha.com – Series[BesselJ[1,x]/x,{x,0,10}] 
    taylor_exp = (1/2)-(bessel_J1**2/16)+(bessel_J1**4/384)-(bessel_J1**6/18432)+(bessel_J1**8/1474560)-(bessel_J1**10/176947200)+(bessel_J1**12/29727129600) 
    
    return 2 * taylor_exp


def reference_pressure(A=0.17, V=18): 
    """
    A — emitter output efficiency (Pa/m*V) 
    V — excitation signal peak-to-peak amplitude (Vpp)  
    """
    return A * V    

def pressure(p_ref, distance, sin_theta, k, r ):
    return p_ref * 1/distance * directivity(sin_theta, k, r) * np.exp(1j*k*distance)


In [35]:

_lambda(40000)
_k(40000)
r = 4.5/1000

z = (0, 0, 0.1) # focal point coordinates [meters]

# list of spatial coordinates of phase array of transducer with hexagonal arrangement (diagonal 3) in meters 
T = hexagon_diagonal_to_coordinates(d=3) 
# print(T)

pressures = [ abs( 
                pressure( 
                    reference_pressure(), 
                        _distance(z, t), 
                        _sin_theta(z, t), 
                        _k(40000), 
                        r ) 
                )   for t in T ]  

# print(*pressures, sep='\n')
print(f'Total pressure: {sum(pressures)} Pa – {pascal_to_db_spl(sum(pressures)) } dB SPL')


Total pressure: 210.58831424823367 Pa – 140.448085448561 dB SPL


In [86]:
z = (0, 0, 0.1) # focal point coordinates [meters]

for n in range(1, 17):
    pressures = [ abs( 
                    pressure( 
                        reference_pressure(A=0.17, V=20), 
                            _distance(z, t), 
                            _sin_theta(z, t), 
                            _k(40000), 
                            r ) 
                        ) for t in hexagon_diagonal_to_coordinates(d=n) ] 
    # print(f'hex({n}) – Total pressure: {sum(pressures)} Pa – {pascal_to_db_spl(sum(pressures)) } dB SPL')  
    print(f'hex({n:02}) – { str(len(hexagon_diagonal_to_coordinates(d=n))).format() } emitters – Total pressure: { round(sum(pressures), 3) } Pa ')  

hex(01) – 1 emitters – Total pressure: 34.0 Pa 
hex(02) – 4 emitters – Total pressure: 134.66 Pa 
hex(03) – 7 emitters – Total pressure: 233.987 Pa 
hex(04) – 14 emitters – Total pressure: 458.76 Pa 
hex(05) – 19 emitters – Total pressure: 615.102 Pa 
hex(06) – 30 emitters – Total pressure: 943.437 Pa 
hex(07) – 37 emitters – Total pressure: 1144.609 Pa 
hex(08) – 52 emitters – Total pressure: 1550.882 Pa 
hex(09) – 61 emitters – Total pressure: 1783.474 Pa 
hex(10) – 80 emitters – Total pressure: 2241.907 Pa 
hex(11) – 91 emitters – Total pressure: 2493.483 Pa 
hex(12) – 114 emitters – Total pressure: 2981.588 Pa 
hex(13) – 127 emitters – Total pressure: 3241.961 Pa 
hex(14) – 154 emitters – Total pressure: 3742.12 Pa 
hex(15) – 169 emitters – Total pressure: 4003.736 Pa 
hex(16) – 200 emitters – Total pressure: 4503.326 Pa 


In [85]:
z = (0, 0, 0.1) # focal point coordinates [meters]

N = 16
pitch = 0.0105 # (m)

for n in range(1, N+1):
    pressures = [ abs( 
                    pressure( 
                        reference_pressure(A=0.17, V=18), 
                            _distance(z, t), 
                            _sin_theta(z, t), 
                            _k(40000), 
                            r ) 
                        ) for t in rectangular_grid_to_coordinates(n, n, pitch) ] 
    
    # print(f'hex({n}) – Total pressure: {sum(pressures)} Pa – {pascal_to_db_spl(sum(pressures)) } dB SPL')  
    print(f'square({n:02}x{n:02}) – { len(rectangular_grid_to_coordinates(n, n, pitch)) } emitters – Total pressure: { sum(pressures) } Pa ')

square(01x01) – 1 emitters – Total pressure: 30.599999999999998 Pa 
square(02x02) – 4 emitters – Total pressure: 121.17312963551996 Pa 
square(03x03) – 9 emitters – Total pressure: 268.18764593222494 Pa 
square(04x04) – 16 emitters – Total pressure: 466.21802007288676 Pa 
square(05x05) – 25 emitters – Total pressure: 708.5254948468678 Pa 
square(06x06) – 36 emitters – Total pressure: 987.6917552701351 Pa 
square(07x07) – 49 emitters – Total pressure: 1296.2017055221681 Pa 
square(08x08) – 64 emitters – Total pressure: 1626.906885417947 Pa 
square(09x09) – 81 emitters – Total pressure: 1973.3420611427823 Pa 
square(10x10) – 100 emitters – Total pressure: 2329.9006130576686 Pa 
square(11x11) – 121 emitters – Total pressure: 2691.894033830969 Pa 
square(12x12) – 144 emitters – Total pressure: 3055.5280365474955 Pa 
square(13x13) – 169 emitters – Total pressure: 3417.8263724303956 Pa 
square(14x14) – 196 emitters – Total pressure: 3776.527675129676 Pa 
square(15x15) – 225 emitters – Total 